#### Normal CNN for the Bird classification

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import zipfile

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
DIR_TRAIN = "\\Users\\Maxi\\Downloads\\archive\\train\\"
DIR_VALID = "\\Users\\Maxi\\Downloads\\archive\\valid\\"
DIR_TEST = "\\Users\\Maxi\\Downloads\\archive\\test\\"

In [26]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_datagen = train_datagen.flow_from_directory(DIR_TRAIN,
                                                target_size=(224,224), 
                                                color_mode='rgb', 
                                                class_mode='categorical', 
                                                batch_size=32,
                                                shuffle=True)

valid_datagen = valid_datagen.flow_from_directory(DIR_VALID,
                                                target_size=(224,224), 
                                                color_mode='rgb', 
                                                class_mode='categorical', 
                                                batch_size=32,
                                                shuffle=True)

test_datagen = test_datagen.flow_from_directory(DIR_TEST,
                                                target_size=(224,224), 
                                                color_mode='rgb', 
                                                class_mode='categorical', 
                                                batch_size=32,
                                                shuffle=True)

Found 56046 images belonging to 385 classes.
Found 1925 images belonging to 385 classes.
Found 1925 images belonging to 385 classes.


In [28]:
def callable_generator(generator):
    def gen():
        for x,y in generator:
            yield x,y
    return gen

train = callable_generator(train_datagen)
valid = callable_generator(valid_datagen)
test = callable_generator(test_datagen)

In [34]:
train_dataset = tf.data.Dataset.from_generator(train, output_types=(tf.float32, tf.float32), output_shapes=((32,224,224,3), (32,)))
valid_dataset = tf.data.Dataset.from_generator(valid, output_types=(tf.float32, tf.float32), output_shapes=((32,224,224,3), (32,)))
test_dataset = tf.data.Dataset.from_generator(test, output_types=(tf.float32, tf.float32), output_shapes=((32,224,224,3), (32,)))

In [35]:
train_dataset

<FlatMapDataset shapes: ((32, 224, 224, 3), (32,)), types: (tf.float32, tf.float32)>

In [55]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.layer1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')
        self.layer2 = tf.keras.layers.MaxPooling2D((2, 2))
        self.layer3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')
        self.layer4 = tf.keras.layers.MaxPooling2D((2, 2))
        self.layer5 = tf.keras.layers.Flatten()
        self.layer6 = tf.keras.layers.Dense(385, activation='softmax')

    def call(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        return x

In [57]:
net = ConvNet()

net.build(input_shape=(32, 224, 224, 3))
net.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [58]:
net.fit(train_datagen, epochs=5, validation_data=valid_datagen)

  ...
    to  
  ['...']
Train for 1752 steps, validate for 61 steps
Epoch 1/5
   1/1752 [..............................] - ETA: 21:44

InvalidArgumentError:  Can not squeeze dim[1], expected a dimension of 1, got 385
	 [[node metrics/accuracy/Squeeze (defined at <ipython-input-58-74d5fb6ad7c4>:1) ]] [Op:__inference_distributed_function_1408]

Function call stack:
distributed_function
